In [1]:
import math

from pathlib import Path
import shutil
from typing import List
import numpy as np

In [ ]:
base_path_string = None

if base_path_string:
    base_path = Path(base_path_string)
    pap = base_path / 'pre_augmented'
    papo = pap / 'original'
    paps = pap / 'segmentations'
    papi = pap / 'images'
    pai = base_path / 'images'
    pam = base_path / 'models'
    pas = base_path / 'segmentations'
    pasu = base_path / 'subsets'

    paths = [pap, papo, paps, papi, pai, pam, pas, pasu]

    for p in paths:
        p.mkdir(parents=True, exist_ok=True)

In [2]:
handpicked_number = 3

In [3]:
from SURE.custom.imageModeConversion import convert_images_in_directory


convert_images_in_directory(f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/original', f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/images', target_mode="RGB")

Processing 1.png: Original mode: L
Converted mode: L
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\1.png
Processing 15.png: Original mode: L
Converted mode: L
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\15.png
Processing 16.png: Original mode: L
Converted mode: L
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\16.png
Processing 17.png: Original mode: L
Converted mode: L
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\17.png
Processing 26.png: Original mode: L
Converted mode: L
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\26.png
Processing 32.png: Original mode: RGBA
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\32.png
Processing 33.png: Original mode: RGBA
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/3/pre_augmented/images\33.png
Processing 46.png: Original mode: I
Converted m

In [4]:
from pathlib import Path
from Core.DataPreparation import AugmentImages
from SURE.custom.paths import empty_directory

def augment_images(input_path: Path, output_path: Path = None, count=20, distort=False):
    
    imagesDirectory = input_path / 'images'
    segmentationsDirectory = input_path / 'segmentations'
    
    if not output_path:
        outputDirectory = input_path.parent
    else:
        outputDirectory = output_path
        
    outputImages = outputDirectory / 'images'
    outputSegmentations = outputDirectory / 'segmentations'
    
    outputImages.mkdir(parents=True, exist_ok=True)
    outputSegmentations.mkdir(parents=True, exist_ok=True)
    
    empty_directory(outputImages)
    empty_directory(outputSegmentations)
    
    AugmentImages(imagesDirectory=imagesDirectory, segmentationsDirectory=segmentationsDirectory, outputDirectory=outputDirectory, count=count, distort=distort)
    
    return (outputImages, outputSegmentations)

In [5]:
images_dir, segmentations_dir, handpicked_path = None, None, None

In [6]:
#@title Generate random images from handpicked examples
from Core.DataPreparation import AugmentImages, SplitData


handpicked_path = Path(f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/')
count = 2000

images_dir, segmentations_dir = augment_images(handpicked_path, count=count)

Images directory: c:\Users\chisa\Documents\GitHub\OrganoID_IIT\SURE\datasets\handpicked\3\pre_augmented\images
Segmentations directory: c:\Users\chisa\Documents\GitHub\OrganoID_IIT\SURE\datasets\handpicked\3\pre_augmented\segmentations
Output directory: c:\Users\chisa\Documents\GitHub\OrganoID_IIT\SURE\datasets\handpicked\3
Initialised with 18 image(s) found.
Output directory set to c:\Users\chisa\Documents\GitHub\OrganoID_IIT\SURE\datasets\handpicked\3.c:\Users\chisa\Documents\GitHub\OrganoID_IIT\SURE\datasets\handpicked\3\pre_augmented\segmentations


18 ground truth image(s) found.


Processing <PIL.Image.Image image mode=I size=512x512 at 0x1D49F54A3B0>: 100%|██████████| 2000/2000 [02:28<00:00, 13.46 Samples/s]  


In [7]:
import math
import shutil
from typing import List

import numpy as np


def _CopyToPath(paths: List[Path], output: Path):
    output.mkdir(parents=True, exist_ok=True)
    for path in paths:
        newPath = output / path.name
        shutil.copy(path, newPath)

def SplitDataMultiple(imagePaths: List[Path], segmentationPaths: List[Path], validationFraction: float,
              testingFraction: float,
              outputDirectory: Path,
              required_numbers: List[int]):
    # Sort paths alphabetically
    imagePaths.sort(key=lambda x: x.stem)
    segmentationPaths.sort(key=lambda x: x.stem)

    # Only use paths that have matched names in image and segmentations directory
    imagePaths = np.asarray([path for path in imagePaths if
                             path.stem in [segPath.stem for segPath in segmentationPaths]])
    segmentationPaths = np.asarray([path for path in segmentationPaths if
                                    path.stem in [imagePath.stem for imagePath in imagePaths]])

    # Carry out the split!
    permutation = np.random.permutation(len(imagePaths))
    
    for n in required_numbers:
        adjusted_number = math.ceil(n/(1-validationFraction-testingFraction))
        current_portion = permutation[:adjusted_number]
        numValidation = math.ceil(adjusted_number * validationFraction)
        
        trainingIndices = current_portion[:n]
        validationIndices = current_portion[n:(n + numValidation)]
        testingIndices = current_portion[(n + numValidation):adjusted_number]

        # Save the images
        _CopyToPath(imagePaths[trainingIndices], outputDirectory / str(n) / "training" / "images")
        _CopyToPath(imagePaths[validationIndices], outputDirectory / str(n) / "validation" / "images")
        _CopyToPath(imagePaths[testingIndices], outputDirectory / str(n) / "testing" / "images")
        _CopyToPath(segmentationPaths[trainingIndices], outputDirectory / str(n) / "training" / "segmentations")
        _CopyToPath(segmentationPaths[validationIndices],
                    outputDirectory / str(n) / "validation" / "segmentations")
        _CopyToPath(segmentationPaths[testingIndices], outputDirectory / str(n) / "testing" / "segmentations")
        

In [8]:
#@title split data
r_path = handpicked_path.parent
if not images_dir or not segmentations_dir or not handpicked_path:
    handpicked_path = Path(f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/')
    r_path = handpicked_path.parent
    images_dir = r_path / 'images'
    segmentations_dir = r_path / 'segmentations'

image_paths = list(images_dir.glob('*.png'))
segmentation_paths = list(segmentations_dir.glob('*.png'))

validation_fraction = 0.15
testing_fraction = 0.15

output_dir = r_path / "subsets"

# empty_directory(output_dir)

output_dir

WindowsPath('SURE/datasets/handpicked/3/subsets')

In [9]:
amounts = list(range(20,1001,20))
amounts

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500,
 520,
 540,
 560,
 580,
 600,
 620,
 640,
 660,
 680,
 700,
 720,
 740,
 760,
 780,
 800,
 820,
 840,
 860,
 880,
 900,
 920,
 940,
 960,
 980,
 1000]

In [10]:
SplitDataMultiple(imagePaths=image_paths, segmentationPaths=segmentation_paths, validationFraction=validation_fraction, testingFraction=testing_fraction, outputDirectory=r_path / "subsets", required_numbers=amounts)

In [11]:
# import tensorflow as tf

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [12]:
# import math
# import pathlib

# import tensorflow as tf
# import numpy as np
# from PIL import Image
# from typing import List, Tuple, Union
# from ImageHandling import NumFrames, GetFrames
# from HelperFunctions import printRep
# from pathlib import Path

